In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re
import time # we'll use this for timing how long things take...

In [145]:
page = "https://www.mangaupdates.com/groups.html?id=8904"
req = requests.get(page)
d_soup = bs(req.content)
OtherInfo = d_soup.find_all("div", "p-1 col-6 text")

In [3]:
def GroupName(d_soup):
    GroupName = d_soup.find_all("div", "p-1 col-6 specialtext")
    return GroupName[0].text

In [4]:
def ReleaseFrequency(OtherInfo):
    ReleaseFrequency = OtherInfo[8].text
    return ReleaseFrequency

In [5]:
def NumberofRelease(OtherInfo):
    NumberofRelease = OtherInfo[10].text
    return NumberofRelease

In [6]:
def Active_or_Not(OtherInfo):
    Active_or_Not = OtherInfo[12].text
    return Active_or_Not

In [7]:
def Total_Series(OtherInfo):
    Total_Series = OtherInfo[14].text
    return Total_Series

In [8]:
def Genres_Sort(OtherInfo):
    Genres = OtherInfo[16].text 
    All_Genres = ['Action','Adult','Adventure','Comedy','Doujinshi','Drama','Ecchi','Fantasy','Gender Bender','Harem','Hentai','Historical','Horror','Josei','Lolicon','Martial Arts','Mature','Mecha','Mystery','Psychological','Romance','School Life','Sci-fi','Seinen','Shotacon','Shoujo','Shoujo Ai','Shounen', 'Shounen Ai','Slice of Life','Smut','Sports','Supernatural','Tragedy','Yaoi','Yuri']
    dict_genres = {}
    genre_name_num = re.findall(r"(.*?)[\s]*[(](.*?)[)][,\s]*",Genres)
    for genre in All_Genres:
        dict_genres[genre] = 0
        for item in genre_name_num:
            if item[0]==genre:
                dict_genres[genre] = item[1]
    return dict_genres


In [9]:
def get_last_updated_day(d_soup):
    if d_soup.find_all("div","col-2 text") == []:
        last_updated_day = "no date"
    else:
        last_updated_day = d_soup.find_all("div", "col-2 text")[0].text
    return last_updated_day

In [184]:
# Get the founding day of the group

def founding_day_new(d_soup, page):
    #IF THE GROUP ONLY HAVE ONE RELEASE PAGE
    if d_soup.find_all("span","d-none") == []:
        if d_soup.find_all("div","col-2 text") == []:
            day = "no date"
        else:

            founding_day_index = len(d_soup.find_all("div","col-2 text")) - 1
            day = d_soup.find_all("div","col-2 text")[founding_day_index].text
    else:
        
    #FIRST, GET THE NUMBER OF THE GROUP'S RELEASE PAGES
        total_page_unclean = d_soup.find_all("span","d-none")[0].text.strip()
        page_series_number = re.findall(r"[0-9]+",total_page_unclean)[0]

        #GO TO THE LAST AND SECOND LAST RELEASE PAGES
        for j in range(len(page)): 
            if page[j] == "?":
                insert = j
        for i in range(int(page_series_number),int(page_series_number)-2,-1):
            turn_page = page[:(insert+1)] + "page={}&".format(i) + page[(insert+1):]

        #TAKE THE FOUNDING DAY (BE CAREFUL SINCE SOME OF DAYS ARE UNKNOWN)
            req_founding_day_page_link = requests.get(turn_page)
            soup_founding_day_page_content = bs(req_founding_day_page_link.content)
            founding_day_init = soup_founding_day_page_content.find_all("div","col-2 text")
            for x in range(len(founding_day_init)-1,0,-1): 
                if founding_day_init[x].text != 'Unknown':
                    day = founding_day_init[x].text
                    break 
        #two break needed to avoid return the last day on the second-last page
            break
    return day


In [11]:
group_dict = {"Name":GroupName(d_soup),"Active or not":Active_or_Not(OtherInfo),"Release Frequency":ReleaseFrequency(OtherInfo),"Number of Release":NumberofRelease(OtherInfo),"Total Series":Total_Series(OtherInfo),"First Update":founding_day_new(d_soup, page),"Lastest Update":get_last_updated_day(d_soup)}

In [12]:
group_dict.update(Genres_Sort(OtherInfo))

In [13]:
group_dict

{'Name': 'Lemonade Scans',
 'Active or not': 'Yes',
 'Release Frequency': 'Often. About every 3 day(s)',
 'Number of Release': '402',
 'Total Series': '74',
 'First Update': '04/13/17',
 'Lastest Update': '06/16/20',
 'Action': 0,
 'Adult': 0,
 'Adventure': 0,
 'Comedy': '18',
 'Doujinshi': 0,
 'Drama': '18',
 'Ecchi': 0,
 'Fantasy': 0,
 'Gender Bender': '1',
 'Harem': '2',
 'Hentai': 0,
 'Historical': 0,
 'Horror': 0,
 'Josei': '4',
 'Lolicon': 0,
 'Martial Arts': 0,
 'Mature': 0,
 'Mecha': 0,
 'Mystery': '1',
 'Psychological': '2',
 'Romance': '64',
 'School Life': '44',
 'Sci-fi': 0,
 'Seinen': 0,
 'Shotacon': 0,
 'Shoujo': '54',
 'Shoujo Ai': 0,
 'Shounen': 0,
 'Shounen Ai': '1',
 'Slice of Life': '16',
 'Smut': '1',
 'Sports': '1',
 'Supernatural': '2',
 'Tragedy': 0,
 'Yaoi': '8',
 'Yuri': 0}

In [14]:
group_cell = pd.DataFrame.from_dict(group_dict, orient='index').transpose()


In [15]:
#Final function to create a group cell

def group_cell(page):
    req = requests.get(page)
    d_soup = bs(req.content)
    OtherInfo = d_soup.find_all("div", "p-1 col-6 text")
    group_dict = {"Name":GroupName(d_soup),"Active or not":Active_or_Not(OtherInfo),"Release Frequency":ReleaseFrequency(OtherInfo),"Number of Release":NumberofRelease(OtherInfo),"Total Series":Total_Series(OtherInfo),"First Update":founding_day_new(d_soup, page),"Lastest Update":get_last_updated_day(d_soup)}
    group_dict.update(Genres_Sort(OtherInfo))
    group_cell = pd.DataFrame.from_dict(group_dict, orient='index').transpose()
    
    return group_cell

SCRAPE THE GROUP PAGE

In [16]:
page_scanlator = 'https://www.mangaupdates.com/groups.html?page=1&'
req_page_scanlator = requests.get(page_scanlator)
soup_page_scanlator = bs(req_page_scanlator.content)
soup_page_scanlator

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html>
<head>
<title>Baka-Updates Manga - Groups</title>
<meta content="Latest And Newest Manga Release Updates and News." name="description"/>
<meta content="manga, anime, updates, update, releases, release, new, scanlators, scanlator, wiki" name="keywords"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="Manick(Code&amp;Html)JYT(Design)" name="author"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="favicon.ico" rel="shortcut icon"/>
<link href="dist/main.26f679a5b9bf274231c0.css" rel="stylesheet" type="text/css"/>
<link href="rss.php" rel="alternate" title="Baka Updates Manga RSS" type="application/rss+xml"/>
<link href="mangaupdates.xml" rel="search" title="MangaUpdates Search" type="application/opensearchdescription+xml"/>
</head>
<body>
<script src="dist/main.35477e88acdffef5682b.js" type="text/javascript"

In [17]:
#scan the page of scanlators 
for i in range(10):
    page_number = i
    page_scanlator_form = 'https://www.mangaupdates.com/groups.html?page={}&'.format(page_number)
    print(page_scanlator_form)

https://www.mangaupdates.com/groups.html?page=0&
https://www.mangaupdates.com/groups.html?page=1&
https://www.mangaupdates.com/groups.html?page=2&
https://www.mangaupdates.com/groups.html?page=3&
https://www.mangaupdates.com/groups.html?page=4&
https://www.mangaupdates.com/groups.html?page=5&
https://www.mangaupdates.com/groups.html?page=6&
https://www.mangaupdates.com/groups.html?page=7&
https://www.mangaupdates.com/groups.html?page=8&
https://www.mangaupdates.com/groups.html?page=9&


In [18]:
#get the link of each scanlator group
scanlator_link_include = soup_page_scanlator.find_all("a", alt="Group Info")
link_group = scanlator_link_include[0].get('href')
link_group

'https://www.mangaupdates.com/groups.html?id=3143'

TESTING THE FIRST PAGE OF GROUP SCANLATION

In [32]:
%%time
df_group_test = pd.DataFrame()
page_scanlator = 'https://www.mangaupdates.com/groups.html?page=2&'
req_page_scanlator = requests.get(page_scanlator)
soup_page_scanlator = bs(req_page_scanlator.content)
scanlator_link_include = soup_page_scanlator.find_all("a", alt="Group Info")
for i in range(len(scanlator_link_include)):
    link_group = scanlator_link_include[i].get('href')
    df_group_test = df_group_test.append(group_cell(link_group))
df_group_test    

Wall time: 21 s


,Name,Active or not,Release Frequency,Number of Release,Total Series,First Update,Lastest Update,Action,Adult,Adventure,...,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Smut,Sports,Supernatural,Tragedy,Yaoi,Yuri
0,/azu/nites,No,Inactive Group,8,1,Unknown,09/23/07,0,0,0,...,0,1,0,1,0,0,0,0,0,0
0,/c/ute scans,No,Inactive Group,7,1,12/24/18,07/24/19,0,0,0,...,0,0,0,1,0,0,1,0,0,0
0,/L/azy Scans,No,Inactive Group,3,1,07/25/19,07/28/19,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,/m/ Scans,No,Inactive Group,2,2,11/23/15,01/24/16,0,0,0,...,2,0,0,0,0,0,0,0,0,0
0,/m/en of Destiny,No,Inactive Group,3,1,Unknown,06/05/15,1,0,0,...,0,1,0,0,0,0,0,0,0,0
0,/m/onkey business,Yes,Occasionally. About every 18 day(s),16,3,11/14/19,06/07/20,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,/r/Bluelock,Yes,Infrequently,2,1,05/19/20,05/22/20,1,0,0,...,0,1,0,0,0,1,0,0,0,0
0,/r/BraveFrontier,No,Inactive Group,25,1,11/08/14,05/03/15,0,0,1,...,0,0,0,0,0,0,0,0,0,0
0,/r/CharlotteAnime,Yes,Infrequently,1,1,12/27/19,12/27/19,0,0,0,...,0,0,0,1,0,0,0,1,0,0
0,/r/manga scans,No,Inactive Group,9,4,08/22/15,08/23/17,2,1,1,...,0,2,0,1,0,0,2,0,0,0


TESTING TWO PAGES AT ONCE

In [33]:
df_group_test_2 = pd.DataFrame()
for i in range(1,3):
    page_number = i
    page_scanlator = 'https://www.mangaupdates.com/groups.html?page={}&'.format(page_number)
    req_page_scanlator = requests.get(page_scanlator)
    soup_page_scanlator = bs(req_page_scanlator.content)
    scanlator_link_all = soup_page_scanlator.find_all("a",alt="Group Info")
    for i in range(len(scanlator_link_all)):
        link_group = scanlator_link_all[i].get('href')
        df_group_test_2 = df_group_test_2.append(group_cell(link_group))
        time.sleep(10)
df_group_test_2

,Name,Active or not,Release Frequency,Number of Release,Total Series,First Update,Lastest Update,Action,Adult,Adventure,...,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Smut,Sports,Supernatural,Tragedy,Yaoi,Yuri
0,#1onlii,No,Inactive Group,9,9,01/03/15,03/22/15,2,1,0,...,0,4,0,2,0,0,3,1,0,0
0,#dropout,Yes,Often. About every 3 to 4 day(s),1018,26,12/10/16,06/17/20,3,0,0,...,0,13,0,10,0,0,8,0,0,0
0,#EverydayHeroes Scans,Yes,Often. About every 6 to 7 day(s),386,12,09/17/15,06/16/20,3,0,4,...,0,0,0,2,0,1,3,0,0,0
0,#got-nyu?,No,Inactive Group,0,0,no date,no date,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,#Hassasin,Yes,Infrequently,4,2,05/23/19,12/30/19,1,0,1,...,0,1,0,0,0,0,0,0,0,0
0,#JustAnotherGroup #BeastScans,Yes,Infrequently,1,1,04/24/20,04/24/20,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,>I Love Girls< Scanlations,No,Inactive Group,3,1,03/23/16,05/09/16,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,<<Infinity ∞ Guild>>,No,Inactive Group,1,1,10/20/18,10/20/18,1,0,1,...,0,0,0,0,0,0,0,0,0,0
0,*Compartilhando*,No,Inactive Group,2,1,09/13/09,09/14/09,1,0,0,...,0,1,0,0,0,0,1,0,0,0
0,*RedSky* Scanlations,No,Inactive Group,2,1,08/23/12,02/14/13,1,0,0,...,0,0,0,1,0,0,0,0,1,0


In [59]:
#One group have problem with release date
page_gotnyu = 'https://www.mangaupdates.com/groups.html?id=10935'
req_page_gotnyu = requests.get(page_gotnyu)
soup_page_gotnyu = bs(req_page_gotnyu.content)
date_haveitornot = soup_page_gotnyu.find_all("div","col-2 text")
date_haveitornot

[]

 TEST TO SEE IF THE ABOVE CODES ARE APPLICABLE TO OTHER GROUPS

In [197]:
page_test = 'https://www.mangaupdates.com/groups.html?id=1665'
req_test = requests.get(page_test)
d_soup_test = bs(req_test.content)
d_soup_test

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html>
<head>
<title>Baka-Updates Manga - ...and other escapades</title>
<meta content="Latest And Newest Manga Release Updates and News." name="description"/>
<meta content="manga, anime, updates, update, releases, release, new, scanlators, scanlator, wiki" name="keywords"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="Manick(Code&amp;Html)JYT(Design)" name="author"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="favicon.ico" rel="shortcut icon"/>
<link href="dist/main.26f679a5b9bf274231c0.css" rel="stylesheet" type="text/css"/>
<link href="rss.php" rel="alternate" title="Baka Updates Manga RSS" type="application/rss+xml"/>
<link href="mangaupdates.xml" rel="search" title="MangaUpdates Search" type="application/opensearchdescription+xml"/>
</head>
<body>
<script src="dist/main.35477e88acdffef5682b.js" type="

In [198]:
GroupName_test = d_soup_test.find_all("div", "p-1 col-6 specialtext")
GroupName_test[0].text

'...and other escapades'

In [199]:
OtherInfo_test = d_soup_test.find_all("div", "p-1 col-6 text")
ReleaseFrequency_test = OtherInfo_test[8].text
ReleaseFrequency_test

'Inactive Group'

In [200]:
Genres_test = OtherInfo_test[16].text
Genres_test

'Doujinshi (2), Yaoi (1), Shounen Ai (1)'

In [201]:
d_soup_test.find_all("div", "col-2 text")[0]

<div class="col-2 text">01/28/08</div>

In [202]:
NumberofRelease_test = OtherInfo_test[10].text
NumberofRelease_test

'2'

In [203]:
d_soup_test.find_all("span","d-none")

[]

In [209]:
if d_soup_test.find_all("span","d-none") == []:
    founding_day_index = len(d_soup_test.find_all("div","col-2 text"))-1
    day = d_soup_test.find_all("div","col-2 text")[founding_day_index].text
day

'01/28/08'

In [220]:
founding_day_new(d_soup_test, page_test)

'01/28/08'

SCRAPE SERIES INFO

In [2]:
series_link = "https://www.mangaupdates.com/series.html?id=1568"
# series_link = "https://www.mangaupdates.com/series.html?id=164002"
request_series_link = requests.get(series_link)
soup_series = bs(request_series_link.content)
soup_series

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html>
<head>
<title>Baka-Updates Manga - Vinland Saga</title>
<meta content="Latest And Newest Manga Release Updates and News." name="description"/>
<meta content="manga, anime, updates, update, releases, release, new, scanlators, scanlator, wiki" name="keywords"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="Manick(Code&amp;Html)JYT(Design)" name="author"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="favicon.ico" rel="shortcut icon"/>
<link href="dist/main.26f679a5b9bf274231c0.css" rel="stylesheet" type="text/css"/>
<link href="rss.php" rel="alternate" title="Baka Updates Manga RSS" type="application/rss+xml"/>
<link href="mangaupdates.xml" rel="search" title="MangaUpdates Search" type="application/opensearchdescription+xml"/>
</head>
<body>
<script src="dist/main.35477e88acdffef5682b.js" type="text/javas

In [3]:
title = soup_series.find_all("span","releasestitle tabletitle")[0].text
title

'Vinland Saga'

In [4]:
comictype = soup_series.find_all("div", "sContent")[1].text.strip()
comictype

'Manga'

In [5]:
def number_group_scanlate(soup_series):
    groups_scanlate = soup_series.find_all("a", title = "Group Info" )
    total_group = []
    for i in groups_scanlate:
        group = i.text
        if group not in total_group:
            total_group.append(group)
    return len(total_group)

In [6]:
number_group_scanlate(soup_series)

14

In [7]:
series_id = re.findall(r'id=([0-9]+)',series_link)

In [8]:
series_id[0]

'1568'

In [15]:
def AwardorNot(series_id):
#find link of the category words in the network tab of developer tool
    category_list_link = "https://www.mangaupdates.com/ajax/show_categories.php?s="+ series_id[0] + "&type=1"
    request_category = requests.get(category_list_link)
    soup_category = bs(request_category.content)
    category_list = soup_category.find_all("li","tag_normal")
    category_list_text = []
    for i in category_list:
        list_category_text.append(i.text)
    if 'Award-Winning Work' or 'Award-Nominated Work' in list_category_text: 
        return True
    else:
        return False
    

In [40]:
#a page that only have release date of every chapter and volumne of a series
release_link = "https://www.mangaupdates.com/releases.html?stype=series&search="+ series_id[0]
# release_link = "https://www.mangaupdates.com/releases.html?search=3106&stype=series"
release_link

'https://www.mangaupdates.com/releases.html?stype=series&search=1568'

In [17]:
%%time
AwardorNot(series_link)

Wall time: 499 ms


True

In [41]:
request_release_link = requests.get(release_link)
release_link_soup = bs(request_release_link.content)


In [42]:
page_number = release_link_soup.find_all("span","d-none")
page_number_only = re.findall(r'[0-9]+',page_number[0].text)[0]
page_number_only = int(page_number_only)
page_number_only

9

In [43]:
#finding the series' scanlation start date
def start_scan_date(page_number_only, release_link):
    for i in range(page_number_only,page_number_only-2,-1):
        release_start_link = release_link + "&page=" + str(i)
        request_release_start = requests.get(release_start_link)
        release_start_soup = bs(request_release_start.content)
        all_date_release = release_start_soup.find_all("div","col-2 text")
        for j in range(len(all_date_release)-1,0,-1):
            if all_date_release[j].text != "Unknown":
                date = all_date_release[j].text 
                break
        break
    return(date)
start_scan_date(page_number_only, release_link)

'08/30/05'

In [44]:
#Start out as scanlation or stimulus translation from the publisher
def publisher_first(release_link,page_number_only):
    last_page = release_link + "&page=" + str(page_number_only)
    request_last_page = requests.get(last_page)
    last_page_soup = bs(request_last_page.content)
    group_name_title = last_page_soup.find_all("div","col-4 text")
    group_name = []
    for i in range(len(group_name_title)):
        if i % 2 == 1:
            group_name.append(group_name_title[i].text)
    
    return "Viz" in group_name or "MANGA Plus" in group_name  
    
publisher_first(release_link,page_number_only)   

False

In [46]:
#Is the series stimulus translated?
def stimulus_translate(release_link_soup):
    title_and_releaser = release_link_soup.find_all("div","col-4 text")
    releaser = []
    for i in range(len(title_and_releaser)): 
        if i % 2 == 1: 
            releaser.append(title_and_releaser[i].text)
    return "Viz" in releaser or "MANGA Plus" in releaser
        


In [77]:
#Genres of the manga
def genres(soup_series):
    All_Genres = ['Action','Adult','Adventure','Comedy','Doujinshi','Drama','Ecchi','Fantasy','Gender Bender','Harem','Hentai','Historical','Horror','Josei','Lolicon','Martial Arts','Mature','Mecha','Mystery','Psychological','Romance','School Life','Sci-fi','Seinen','Shotacon','Shoujo','Shoujo Ai','Shounen', 'Shounen Ai','Slice of Life','Smut','Sports','Supernatural','Tragedy','Yaoi','Yuri']
    all_sContent = soup_series.find_all("div","sContent")
    genres_series = all_sContent[14]
    genres_series_only = genres_series.find_all("u")
    genres_series_list = []
    genres_series_dict = {}
    for i in range(len(genres_series_only)-1): 
        genres_series_list.append(genres_series_only[i].text)
    for x in All_Genres: 
        if x in genres_series_list: 
            genres_series_dict[x] = 1
        else: 
            genres_series_dict[x] = 0
    return genres_series_dict
    
    